File Field Example
----------------
This File Field example application allows you to create simple file uploader


In [ ]:
from bspump.jupyter import *
from bspump.http.web.server import *
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import io
from datetime import date

STOCKS = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "JPM", "NFLX", "AMD"]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

In [ ]:
def fetch_yahoo_price(ticker, start_date, end_date):
    start_ts = int(time.mktime(start_date.timetuple()))
    end_ts = int(time.mktime(end_date.timetuple()))

    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}"
    params = {
        "period1": start_ts,
        "period2": end_ts,
        "interval": "1d"
    }

    try:
        resp = requests.get(url, params=params, headers=HEADERS)
        data = resp.json()

        result = data["chart"]["result"][0]
        timestamps = result["timestamp"]
        closes = result["indicators"]["quote"][0]["close"]

        dates = [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamps]
        return pd.Series(closes, index=dates)
    except Exception as e:
        print(f"Error for {ticker}: {e}")
        return None


def get_price_changes(tickers, date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    day_before = date_obj - timedelta(days=1)
    day_after = date_obj + timedelta(days=1)

    changes = {}
    for ticker in tickers:
        series = fetch_yahoo_price(ticker, day_before, day_after)
        print(series)
        if series is not None and date_str in series and day_before.strftime("%Y-%m-%d") in series:
            prev = series[day_before.strftime("%Y-%m-%d")]
            curr = series[date_str]
            if prev and curr:
                change = ((curr - prev) / prev) * 100
                changes[ticker] = change
    return changes

def plot_with_pandas(changes, date_str):
    df = pd.DataFrame(list(changes.items()), columns=["Ticker", "Change (%)"])
    df.sort_values("Change (%)", ascending=False, inplace=True)

    ax = df.plot.bar(
        x="Ticker",
        y="Change (%)",
        title=f"Stock Change (%) on {date_str}",
        xlabel="Ticker",
        ylabel="Change (%)",
        grid=True,
        figsize=(10, 5)
    )

    buf = io.BytesIO()
    ax.figure.savefig(buf, format="jpeg")
    return buf


Then we define a simple web form to get the File we want to upload

In [ ]:
auto_pipeline(
    source=lambda app, pipeline: WebFormSource(app, pipeline, route="/",
    form_intro="""<p> Select date range and click submit to see a visualization""",
    fields=[
        IntField("Day",default=20),
        IntField("Month",default=2),
        IntField("Year",default=2025),
    ]),
    sink=lambda app, pipeline: WebSink(app, pipeline)
)

Pipeline section
----------------

Everything after this is rerun every time an event comes in. At run time, the `event` variable is automatically set with the value of the event that comes from the source.

We can do whatever transformations we please, and then, by setting `event` at the end of the notebook, the value of `event` will automatically be sent to the sink.

In [ ]:
form = event["form"]
parsed_date = date(form["Year"], form["Month"], form["Day"]).strftime("%Y-%m-%d")
try:
    changes = get_price_changes(STOCKS, parsed_date)
    if not changes:
        print("⚠️ No data found. Market may have been closed.")
        return
    top_stock = max(changes, key=changes.get)
    print(f"📈 Most increasing stock on {parsed_date}: {top_stock} ({changes[top_stock]:.2f}%)")
    img_data = plot_with_pandas(changes, parsed_date)
    event["response"] = img_data.getvalue()
    event["content_type"] = "image/jpeg"

except Exception as e:
    print(f"❌ Error: {e}")
